Imports:

In [1]:
import torch
from torch import nn  # All neural network modules
from torch import optim  # For optimizers like SGD, Adam, etc.
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment by creating mini batches etc.
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation

Define hyperparameters:

In [2]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5  # increase to boost performance

Set device:

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

We define a RNN architecture:

The input is of shape $b \times 1 \times 28 \times 28$. We can view it as $b$ datapoints, each made of $28$ time sequences and each time sequence is composed of $28$ features.

In [4]:
class RNN(nn.Module):
    # input_size: num of features for each timestamp
    # hidden_size: the number of features in the hidden state h
    # num_layers: num of layers for the RNN
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    # input x has shape (batch_size, sequence_length, input_size)
    def forward(self, x):
        # h0 has to be tensor of shape (num_layers, hidden_size) for unbatched input
        # or (num_layers, batch_size, hidden_size) containing the initial hidden state for the input sequence batch
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.gru(x, h0)
        # out has shape (batch_size, sequence_length, hidden_size)
        # flatten out to fit in FC NN
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [5]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5  # increase to boost performance

Load Data:

In [6]:
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Initialize our NN:

In [7]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)  # we set the initializer default parameters

Loss function and optimization function:

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train the NN:

In [9]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to CUDA if possible
        # convert MNIST data from shape (bx1x28x28) to (bx28x28)
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward pass
        pred = model(data)

        # evaluate the loss
        loss = criterion(pred, targets)

        # backward
        # Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
        optimizer.zero_grad()  # set all gradient to zero for each batch
        # Backpropagate the prediction loss with a call to loss.backward().
        # PyTorch deposits the gradients of the loss w.r.t. each parameter.
        loss.backward()  # compute the gradient

        # gradient descent or backward step
        # Performs a single optimization step (parameter update).
        optimizer.step()

Check accuracy on training and testing dataset:

In [10]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking accuracy on training data")
    else:
        print("checking accuracy on testing data")

    num_correct = 0
    num_samples = 0
    # TODO
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            res = model(x)
            # the predictions are the indices
            _, pred = res.max(1)
            num_correct += (y == pred).sum()
            num_samples += pred.shape[0]
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/ num_samples * 100:.2f}")

In [11]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking accuracy on training data
Got 59561 / 60000 with accuracy 99.27
checking accuracy on testing data
Got 9877 / 10000 with accuracy 98.77
